In [3]:
# coding: utf-8
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os

import pandas as pd

import h1st as h1
h1.init()

from examples.RayTune.sklearn_smv_classifier import SklearnSVMClassifier
from examples.RayTune.tensorflow_mlp_classifier import TensorflowMLPClassifier
from examples.RayTune import config
from examples.RayTune.utils import prepare_train_test_data

## Tune's Functional API

In [5]:
import ray
from ray import tune

In [6]:
ray.init(ignore_reinit_error=True)

2020-10-23 15:49:55,171	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.4.20',
 'raylet_ip_address': '192.168.4.20',
 'redis_address': '192.168.4.20:6379',
 'object_store_address': '/tmp/ray/session_2020-10-23_15-49-54_664484_16984/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-10-23_15-49-54_664484_16984/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2020-10-23_15-49-54_664484_16984',
 'metrics_export_port': 64137}

In [9]:
config = {
    "lr": tune.grid_search([0.005]),
    "units": tune.grid_search([16, 32, 64, 128]),
    "n_layer": tune.grid_search([3, 5, 10])
}

In [10]:
DATA_ROOT = './data/credit_card'

def train_model_with_tune(config):
    from ray.tune import report
    import os
    from filelock import FileLock    
    h1_tf_mlp = TensorflowMLPClassifier(
        units=config['units'], lr=config['lr'], n_layer=config['n_layer'])
    
    lock_file = f'{DATA_ROOT}/data.lock'
    if not os.path.exists(DATA_ROOT):
        os.makedirs(DATA_ROOT)
    with FileLock(os.path.expanduser(lock_file)):
        data = h1_tf_mlp.load_data()
    prepared_data = h1_tf_mlp.prep(data) 
 
    h1_tf_mlp.train(prepared_data)
    h1_tf_mlp.evaluate(prepared_data)
    for _ in range(5):
        report(iterations=100, mean_accuracy=h1_tf_mlp.metrics['accuracy'])    

#     h1_tf_mlp.persist('my_tf_mlp')
#     report(mean_accuracy=h1_tf_mlp.metrics['accuracy'])

#     train_loader, test_loader = get_data_loaders()
#     model = ConvNet()
#     optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config['momentum'])
#     for i in range(10):
#         train(model, optimizer, train_loader)
#         acc = test(model, test_loader)
        

In [11]:
analysis_func = tune.run(train_model_with_tune, config=config, stop={"training_iteration": 10})

Trial name,status,loc,lr,n_layer,units
train_model_with_tune_0c363_00000,RUNNING,,0.005,3,16
train_model_with_tune_0c363_00001,PENDING,,0.005,5,16
train_model_with_tune_0c363_00002,PENDING,,0.005,10,16
train_model_with_tune_0c363_00003,PENDING,,0.005,3,32
train_model_with_tune_0c363_00004,PENDING,,0.005,5,32
train_model_with_tune_0c363_00005,PENDING,,0.005,10,32
train_model_with_tune_0c363_00006,PENDING,,0.005,3,64
train_model_with_tune_0c363_00007,PENDING,,0.005,5,64
train_model_with_tune_0c363_00008,PENDING,,0.005,10,64
train_model_with_tune_0c363_00009,PENDING,,0.005,3,128


7, in _train
    self._report_thread_runner_error(block=True)
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 262, in _report_thread_runner_error
    .format(err_tb_str)))
ray.tune.error.TuneError: Trial raised an exception. Traceback:
ray::ImplicitFunc.train() (pid=14462, ip=192.168.4.20)
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 109, in run
    self._entrypoint()
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 163, in entrypoint
    return self._trainable_func(config, self._status_reporter)
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 273, in _trainable_func
    output = train_func(config)
  File "<ipython-input-10-56c1d84d2ea4>", line 8, in train_model_with_tune
TypeError: __init__() missing 1 required positional argum

Trial name,status,loc,lr,n_layer,units
train_model_with_tune_0c363_00000,RUNNING,,0.005,3,16
train_model_with_tune_0c363_00001,RUNNING,,0.005,5,16
train_model_with_tune_0c363_00002,RUNNING,,0.005,10,16
train_model_with_tune_0c363_00003,RUNNING,,0.005,3,32
train_model_with_tune_0c363_00004,RUNNING,,0.005,5,32
train_model_with_tune_0c363_00005,ERROR,,0.005,10,32
train_model_with_tune_0c363_00006,RUNNING,,0.005,3,64
train_model_with_tune_0c363_00007,RUNNING,,0.005,5,64
train_model_with_tune_0c363_00008,PENDING,,0.005,10,64
train_model_with_tune_0c363_00009,PENDING,,0.005,3,128


eof_cause()
ray.exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train() (pid=14455, ip=192.168.4.20)
  File "python/ray/_raylet.pyx", line 446, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 400, in ray._raylet.execute_task.function_executor
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/trainable.py", line 261, in train
    result = self._train()
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 217, in _train
    self._report_thread_runner_error(block=True)
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 262, in _report_thread_runner_error
    .format(err_tb_str)))
ray.tune.error.TuneError: Trial raised an exception. Traceback:
ray::ImplicitFunc.train() (pid=14455, ip=192.168.4.20)
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/function_runner.py", line 1

Trial name,status,loc,lr,n_layer,units
train_model_with_tune_0c363_00000,ERROR,,0.005,3,16
train_model_with_tune_0c363_00001,ERROR,,0.005,5,16
train_model_with_tune_0c363_00002,ERROR,,0.005,10,16
train_model_with_tune_0c363_00003,ERROR,,0.005,3,32
train_model_with_tune_0c363_00004,ERROR,,0.005,5,32
train_model_with_tune_0c363_00005,ERROR,,0.005,10,32
train_model_with_tune_0c363_00006,ERROR,,0.005,3,64
train_model_with_tune_0c363_00007,ERROR,,0.005,5,64
train_model_with_tune_0c363_00008,RUNNING,,0.005,10,64
train_model_with_tune_0c363_00009,RUNNING,,0.005,3,128


2020-10-23 14:52:51,047	ERROR trial_runner.py:520 -- Trial train_model_with_tune_0c363_00010: Error processing event.
Traceback (most recent call last):
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 468, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 430, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python3.7/site-packages/ray/worker.py", line 1474, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(TuneError): ray::ImplicitFunc.train() (pid=14514, ip=192.168.4.20)
  File "python/ray/_raylet.pyx", line 446, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 400, in ray._raylet.execute_task.function_executor
  File "/Users/baonguyen/Projects/arimo/h1st/venv/lib/python

Trial name,status,loc,lr,n_layer,units
train_model_with_tune_0c363_00000,ERROR,,0.005,3,16
train_model_with_tune_0c363_00001,ERROR,,0.005,5,16
train_model_with_tune_0c363_00002,ERROR,,0.005,10,16
train_model_with_tune_0c363_00003,ERROR,,0.005,3,32
train_model_with_tune_0c363_00004,ERROR,,0.005,5,32
train_model_with_tune_0c363_00005,ERROR,,0.005,10,32
train_model_with_tune_0c363_00006,ERROR,,0.005,3,64
train_model_with_tune_0c363_00007,ERROR,,0.005,5,64
train_model_with_tune_0c363_00008,ERROR,,0.005,10,64
train_model_with_tune_0c363_00009,ERROR,,0.005,3,128


TuneError: ('Trials did not complete', [train_model_with_tune_0c363_00000, train_model_with_tune_0c363_00001, train_model_with_tune_0c363_00002, train_model_with_tune_0c363_00003, train_model_with_tune_0c363_00004, train_model_with_tune_0c363_00005, train_model_with_tune_0c363_00006, train_model_with_tune_0c363_00007, train_model_with_tune_0c363_00008, train_model_with_tune_0c363_00009, train_model_with_tune_0c363_00010, train_model_with_tune_0c363_00011])

In [ ]:
print("Best config: ", analysis_func.get_best_config(metric="mean_accuracy"))

In [ ]:
analysis_func.dataframe().sort_values('mean_accuracy', ascending=False).head()

In [ ]:
analysis_func.dataframe()[['mean_accuracy', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

In [ ]:
stats = analysis_func.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
ray.shutdown()

In [ ]:
# h1_tf_mlp = TensorflowMLPClassifier(units=32, lr=0.01)
# data = h1_tf_mlp.load_data()
# prepared_data = h1_tf_mlp.prep(data)
# h1_tf_mlp.train(prepared_data)
# h1_tf_mlp.evaluate(prepared_data)
# h1_tf_mlp.metrics
# h1_tf_mlp.persist('my_tf_mlp')

## Tune's Class API

In [ ]:
ray.init(ignore_reinit_error=True)

In [5]:
class TrainH1stModel(tune.Trainable):
    def _setup(self, config):
        from ray.tune import report
        import os
        from filelock import FileLock            
        self.hyper_parameters = {
            'units': config['units'], 'n_layer': config['n_layer']}
        self.h1_tf_mlp = TensorflowMLPClassifier(lr=0.01, config['units'],  config['n_layer'])
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_tf_mlp.load_data()
        self.prepared_data = self.h1_tf_mlp.prep(data)         
        
    def _train(self):
        self.h1_tf_mlp.train(self.prepared_data)
        self.h1_tf_mlp.evaluate(self.prepared_data)
        acc = self.h1_tf_mlp.metrics['accuracy']
        return {"mean_accuracy": acc}

SyntaxError: positional argument follows keyword argument (<ipython-input-5-8a96cf09a997>, line 8)

In [ ]:
config = {
    "lr": tune.grid_search([0.005]),
    "units": tune.grid_search([16, 32, 64, 128]),
    "n_layer": tune.grid_search([3, 5, 10])
#     "n_layer": tune.uniform(1, 10)
}
 
analysis = tune.run(
    TrainH1stModel, 
    config=config, 
    verbose=2,
    stop={"training_iteration": 3}
)

## Population Base Training

In [7]:
# common

import json
import os
import datetime

class TrainH1stModel(tune.Trainable):
    def _setup(self, config):
        from filelock import FileLock 
        self.config = config
        self.timestep = 0
        self.h1_tf_mlp = TensorflowMLPClassifier(
            units=int(config['units']), 
            epochs=int(config['epochs']), 
            lr=config['lr'], 
            n_layer=int(config['n_layer']))
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_tf_mlp.load_data()
        self.prepared_data = self.h1_tf_mlp.prep(data)         
        
    def _train(self):
        self.timestep += 1
        self.h1_tf_mlp.train(self.prepared_data)
        self.h1_tf_mlp.evaluate(self.prepared_data)

        # tf_mlp_{timestep}_{lr}_{layer}_{unit}_{epochs}_{ts}
        version_name = "tf_mlp_" + (
            "_".join(f"{k}:{v}" for k, v in self.config.items())
        ) + "_" + str(int(datetime.datetime.utcnow().timestamp()))

        version = self.h1_tf_mlp.persist(version_name)
        acc = self.h1_tf_mlp.metrics['accuracy']
        return {"mean_accuracy": acc, 'model_version': version}
    
    def _save(self, checkpoint_dir):
        import json
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def _restore(self, checkpoint_path):
        import json
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]

In [14]:
# XXX 2

from ray.tune.schedulers import PopulationBasedTraining

experiment_metrics = dict(metric="mean_accuracy", mode="max")

pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        perturbation_interval=2,  # Every N time_attr units, "perturb" the parameters.
        hyperparam_mutations={           
            "lr": [0.005],
            "epochs": [2, 10, 30, 100],
            "units": [4, 8, 16, 32, 64],
            "n_layer": [2, 4, 6]            
        },
        **experiment_metrics)

# Note: This appears to be needed to avoid a "key error", but in fact these values won't change
# in the analysis.dataframe() object, even though they will be tuned by the PBT scheduler.
# So when you look at the analysis.dataframe(), look at the `experiment_tag` to see the actual values!
config = {
    "lr": 0.005,            # Use the lowest values from the previous definition
    "units": 16,
    "n_layer": 4,
    "epochs": 2
}

In [16]:
# XXX 3

analysis = tune.run(TrainH1stModel, 
    scheduler=pbt_scheduler, 
    config=config,
    stop={"training_iteration": 4},
#     stop={"mean_accuracy": 0.82, "training_iteration": 200},                    
    num_samples=40,
    verbose=1
)

stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

Trial name,status,loc,acc,iter,total time (s)
TrainH1stModel_9ac64_00000,TERMINATED,,0.812222,4,7.85084
TrainH1stModel_9ac64_00001,TERMINATED,,0.810505,4,9.03762
TrainH1stModel_9ac64_00002,TERMINATED,,0.812828,4,9.21953
TrainH1stModel_9ac64_00003,TERMINATED,,0.810808,4,9.93408
TrainH1stModel_9ac64_00004,TERMINATED,,0.813838,4,9.49822
TrainH1stModel_9ac64_00005,TERMINATED,,0.815758,4,11.2503
TrainH1stModel_9ac64_00006,TERMINATED,,0.807273,4,9.70593
TrainH1stModel_9ac64_00007,TERMINATED,,0.816162,4,10.6315
TrainH1stModel_9ac64_00008,TERMINATED,,0.811919,4,8.45657
TrainH1stModel_9ac64_00009,TERMINATED,,0.806061,4,8.75408


 315.74 seconds,    5.26 minutes


In [ ]:
analysis.dataframe()[['trial_id', 'mean_accuracy', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

## BO Hyperband (BOHB)

In [6]:
# import json
# import os

# import numpy as np

# import ray
# from ray import tune
# from ray.tune import Trainable
# from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
# from ray.tune.suggest.bohb import TuneBOHB


# class MyTrainableClass(Trainable):
#     """Example agent whose learning curve is a random sigmoid.

#     The dummy hyperparameters "width" and "height" determine the slope and
#     maximum reward value reached.
#     """

#     def setup(self, config):
#         self.timestep = 0

#     def step(self):
#         self.timestep += 1
#         v = np.tanh(float(self.timestep) / self.config.get("width", 1))
#         v *= self.config.get("height", 1)

#         # Here we use `episode_reward_mean`, but you can also report other
#         # objectives such as loss or accuracy.
#         return {"episode_reward_mean": v}

#     def save_checkpoint(self, checkpoint_dir):
#         path = os.path.join(checkpoint_dir, "checkpoint")
#         with open(path, "w") as f:
#             f.write(json.dumps({"timestep": self.timestep}))
#         return path

#     def load_checkpoint(self, checkpoint_path):
#         with open(checkpoint_path) as f:
#             self.timestep = json.loads(f.read())["timestep"]


# if __name__ == "__main__":
#     import ConfigSpace as CS  # noqa: F401
#     ray.init(num_cpus=8)

#     config = {
#         "iterations": 100,
#         "width": tune.uniform(0, 20),
#         "height": tune.uniform(-100, 100),
#         "activation": tune.choice(["relu", "tanh"])
#     }

#     # Optional: Pass the parameter space yourself
#     # config_space = CS.ConfigurationSpace()
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("width", lower=0, upper=20))
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("height", lower=-100, upper=100))
#     # config_space.add_hyperparameter(
#     #     CS.CategoricalHyperparameter(
#     #         "activation", choices=["relu", "tanh"]))

#     experiment_metrics = dict(metric="episode_reward_mean", mode="max")

#     bohb_hyperband = HyperBandForBOHB(
#         time_attr="training_iteration",
#         max_t=100,
#         reduction_factor=4,
#         **experiment_metrics)

#     bohb_search = TuneBOHB(
#         # space=config_space,  # If you want to set the space manually
#         max_concurrent=4,
#         **experiment_metrics)

#     tune.run(
#         MyTrainableClass,
#         name="bohb_test",
#         config=config,
#         scheduler=bohb_hyperband,
#         search_alg=bohb_search,
#         num_samples=10,
#         stop={"training_iteration": 100})

In [7]:
import ConfigSpace as CS
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB

In [8]:
config_space = CS.ConfigurationSpace()

# There are also UniformIntegerHyperparameter and UniformFloatHyperparameter
# objects for defining integer and float ranges, respectively. For example:
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('foo', lower=0, upper=100))

# config_space.add_hyperparameter(
#     CS.UniformFloatHyperparameter('lr', lower=0.001, upper=0.1))
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('units', lower=1, upper=64))
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('n_layer', lower=1, upper=8))

config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('lr', choices=[0.005]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('units', choices=[4, 8, 16, 32]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('n_layer', choices=[2, 4, 6, 8]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('epochs', choices=[2, 10, 30, 100]))

epochs, Type: Categorical, Choices: {2, 10, 30, 100}, Default: 2

In [9]:
experiment_metrics = dict(metric="mean_accuracy", mode="max")
search_algorithm = TuneBOHB(config_space, max_concurrent=4, **experiment_metrics)
# search_algorithm = TuneBOHB(config_space, max_concurrent=4)

scheduler = HyperBandForBOHB(
    time_attr='training_iteration',
    reduction_factor=4,)
#     **experiment_metrics)

In [14]:
analysis = tune.run(
    TrainH1stModel, 
    scheduler=scheduler, 
    search_alg=search_algorithm, 
    metric="mean_accuracy",
    mode="max",         
    num_samples=40,                           # Force it try all 12 combinations
    stop={"training_iteration": 4},
    verbose=1
)

NameError: name 'scheduler' is not defined

In [11]:
analysis.dataframe()[['mean_accuracy', 'config/epochs','config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

,mean_accuracy,config/epochs,config/lr,config/n_layer,config/units
9,0.820505,100,0.005,4,4
10,0.820000,100,0.005,2,4
14,0.819091,30,0.005,2,32
23,0.819091,30,0.005,4,32
11,0.818889,10,0.005,8,8
24,0.818889,10,0.005,8,32
22,0.818687,100,0.005,2,16
4,0.818687,10,0.005,4,32
35,0.818384,100,0.005,8,16
2,0.817980,10,0.005,4,8


## Bayesian Optimization (BO)

In [8]:
"""This test checks that BayesOpt is functional.

It also checks that it is usable with a separate scheduler.
"""
import time


import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch


# def evaluation_fn(step, width, height):
#     return (0.1 + width * step / 100)**(-1) + height * 0.1


# def easy_objective(config):
#     # Hyperparameters
#     width, height = config["width"], config["height"]

#     for step in range(config["steps"]):
#         # Iterative training function - can be any arbitrary training procedure
#         intermediate_score = evaluation_fn(step, width, height)
#         # Feed the score back back to Tune.
#         tune.report(iterations=step, mean_loss=intermediate_score)
#         time.sleep(0.1)

space = {
    "lr": tune.uniform(0.005, 0.05),
    "units": tune.quniform(1, 64, 1),
#     "units": tune.randint(1, 64),
#     "n_layer": tune.randint(1, 8)
    "n_layer": tune.quniform(1, 8, 1),    
    "epochs": tune.quniform(1, 50, 1),
}

algo = BayesOptSearch(   
    utility_kwargs={
        "kind": "ucb",
        "kappa": 2.5,
        "xi": 0.0
    }
)

algo = ConcurrencyLimiter(algo, max_concurrent=4)
# scheduler = AsyncHyperBandScheduler()
analysis= tune.run(
    TrainH1stModel,
    config=space,
    metric="mean_accuracy",
    mode="max",     
#     name="my_exp",
    search_alg=algo,
#     scheduler=scheduler,
    stop={"training_iteration": 4},
    verbose=1,
    num_samples=20,
)


Trial name,status,loc,epochs,lr,n_layer,units,acc,iter,total time (s)
TrainH1stModel_20af0fa6,TERMINATED,,19.3525,0.0477821,6.12396,38.7155,0.774747,4,15.0799
TrainH1stModel_20afdb84,TERMINATED,,8.64491,0.0120198,1.40659,55.5691,0.806364,4,5.86382
TrainH1stModel_20b09358,TERMINATED,,30.4546,0.0368633,1.14409,62.1043,0.805253,4,7.91673
TrainH1stModel_20b1343e,TERMINATED,,41.7897,0.0145553,2.27277,12.5545,0.818788,4,11.3039
TrainH1stModel_2b287ba2,TERMINATED,,15.9079,0.028614,4.02362,19.3474,0.817778,4,10.2514
TrainH1stModel_2c7ff55c,TERMINATED,,30.9808,0.0112772,3.04501,24.0808,0.818081,4,11.5528
TrainH1stModel_2e8af964,TERMINATED,,23.3474,0.0403329,2.39772,33.3968,0.813636,4,10.2772
TrainH1stModel_30d6033a,TERMINATED,,30.0283,0.00709027,5.25281,11.743,0.816364,4,10.3606
TrainH1stModel_3aa84882,TERMINATED,,4.18753,0.0476998,7.75942,51.929,0.774747,4,6.2738
TrainH1stModel_3ba4cdb4,TERMINATED,,15.9261,0.00939525,5.78963,28.7296,0.817576,4,6.47287


In [9]:
analysis.dataframe()[0:10]

,mean_accuracy,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_this_iter_s,time_total_s,...,time_since_restore,timesteps_since_restore,iterations_since_restore,trial_id,experiment_tag,config/epochs,config/lr,config/n_layer,config/units,logdir
0,0.817778,False,NaN,NaN,1,ec389e0f5dad45b5aad315fef8591f33,2020-10-23_15-21-42,1603491702,3.595754,3.595754,...,3.595754,0,1,183b5fb8,"1_epochs=19.352,lr=0.047782,n_layer=6.124,unit...",19.352466,0.047782,6.123958,38.715485,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
1,0.805758,True,NaN,NaN,4,5d03422a3abc4fcbb988a9afef89bbdc,2020-10-23_15-21-44,1603491704,1.358710,5.534624,...,5.534624,0,4,183bf96e,"2_epochs=8.6449,lr=0.01202,n_layer=1.4066,unit...",8.644913,0.012020,1.406585,55.569097,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
2,0.812525,True,NaN,NaN,4,46cacac6147941029c5ab98456b51c15,2020-10-23_15-21-46,1603491706,1.777796,7.660266,...,7.660266,0,4,183c7074,"3_epochs=30.455,lr=0.036863,n_layer=1.1441,uni...",30.454636,0.036863,1.144091,62.104321,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
3,0.817475,False,NaN,NaN,3,5c05b20c3ffe4a0bb39a94f9c489e98f,2020-10-23_15-21-46,1603491706,2.602008,7.580446,...,7.580446,0,3,183ce5a4,"4_epochs=41.79,lr=0.014555,n_layer=2.2728,unit...",41.789689,0.014555,2.272775,12.554484,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
4,0.817778,False,NaN,NaN,2,4e11fc575bda4cb080b7a27d73594231,2020-10-23_15-22-00,1603491720,2.097678,3.643604,...,3.643604,0,2,22234c66,"5_epochs=15.908,lr=0.028614,n_layer=4.0236,uni...",15.907870,0.028614,4.023615,19.347436,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
5,0.817576,False,NaN,NaN,1,e27682e3b9eb4376b0ca347d9e423fe2,2020-10-23_15-21-59,1603491719,2.124254,2.124254,...,2.124254,0,1,235c08fc,"6_epochs=30.981,lr=0.011277,n_layer=3.045,unit...",30.980792,0.011277,3.045013,24.080796,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
6,0.818081,False,NaN,NaN,1,2f6f509ebe8b4decbb7df595b5a2ab2f,2020-10-23_15-22-00,1603491720,2.250023,2.250023,...,2.250023,0,1,247dcf9a,"7_epochs=23.347,lr=0.040333,n_layer=2.3977,uni...",23.347429,0.040333,2.397716,33.396770,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
7,0.818990,False,NaN,NaN,2,0f74edfb04534843a66f7244445e34a8,2020-10-23_15-22-09,1603491729,3.218707,6.821866,...,6.821866,0,2,26373ec0,"8_epochs=30.028,lr=0.0070903,n_layer=5.2528,un...",30.028314,0.007090,5.252814,11.743020,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
8,0.775859,False,NaN,NaN,1,a67f91e10de94c77bfa259a9a635bb3e,2020-10-23_15-22-19,1603491739,1.591708,1.591708,...,1.591708,0,1,2e767fe2,"9_epochs=4.1875,lr=0.0477,n_layer=7.7594,units...",4.187528,0.047700,7.759424,51.929033,/Users/baonguyen/ray_results/TrainH1stModel/Tr...
9,0.817576,False,NaN,NaN,1,b7f13aaca0354c7abb9f5970b56af239,2020-10-23_15-22-19,1603491739,1.593390,1.593390,...,1.593390,0,1,2f474c08,"10_epochs=15.926,lr=0.0093952,n_layer=5.7896,u...",15.926075,0.009395,5.789631,28.729607,/Users/baonguyen/ray_results/TrainH1stModel/Tr...


In [8]:
analysis.dataframe()[['mean_accuracy', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

,mean_accuracy,config/lr,config/n_layer,config/units
35,0.820000,0.038323,2.814838,20.963484
15,0.819798,0.039983,4.890566,33.750592
19,0.819394,0.028303,3.894907,23.041687
26,0.819394,0.050000,2.522076,26.935785
29,0.819293,0.038700,3.448717,32.467524
37,0.819192,0.034986,2.620923,22.334667
33,0.819192,0.033853,2.693100,24.183756
25,0.819091,0.030983,2.150246,27.382300
32,0.819091,0.050000,5.626195,34.432538
18,0.818990,0.018124,3.265483,35.539160


## OptunaSearch

## HyperOptSearch